### Imports ###

In [ ]:
import socket
import xml.etree.ElementTree as ET

In [ ]:
import pybullet as p

import pybullet as p
import pybullet_data
import math
import time

# ------------------ Setup ------------------
p.connect(p.GUI)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.setGravity(0, 0, -9.81)

plane = p.loadURDF("plane.urdf")

urdf_path = r"/home/lukas/Dokumente/KRL2Python/kuka\kuka_kr3_support\urdf\kr3r540.urdf"
robot = p.loadURDF(urdf_path, useFixedBase=True)

# ------------------ Reale KUKA-Gelenkwinkel ------------------
real_joint_angles_deg = [0.0, -90.0, 0.0, 0.0, 0.0, 45.0]
real_joint_angles_rad = [math.radians(a) for a in real_joint_angles_deg]

# Roboter initial setzen
for j in range(6):
    p.resetJointState(robot, j, real_joint_angles_rad[j])

# ------------------ Slider-Grenzen (Grad) ------------------
joint_limits_deg = [
    (-185, 185),   # A1
    (-120, 120),   # A2
    (-140, 140),   # A3
    (-350, 350),   # A4
    (-120, 120),   # A5
    (-350, 350)    # A6
]

# ------------------ Slider erzeugen (Initialwert = reale Winkel) ------------------
slider_ids = []
for i, (lo, hi) in enumerate(joint_limits_deg):
    slider = p.addUserDebugParameter(
        f"A{i+1} (deg)",
        lo,
        hi,
        real_joint_angles_deg[i]  # <<< HIER der Trick
    )
    slider_ids.append(slider)

# ------------------ Simulationsloop ------------------
while True:
    p.stepSimulation()

    # Slider lesen (Grad)
    deg_values = [p.readUserDebugParameter(s) for s in slider_ids]

    # In Radiant umrechnen
    rad_values = [math.radians(d) for d in deg_values]

    # Roboter steuern
    p.setJointMotorControlArray(
        robot,
        list(range(6)),
        p.POSITION_CONTROL,
        targetPositions=rad_values
    )

    time.sleep(1/240)

AttributeError: module 'pybullet' has no attribute '_version_'

### class MovePrimitve ###
- Represents one robot motion command
- Holds motion parameters:
    - Type, Mode, BaseIndex, ToolIndex, Velocity
    - X, Y, Z, A, B, C (target pose)

- Provides method to_xml() → converts data into valid XML message

In [2]:
class MovePrimitive:
    def __init__(
        self,
        cmd_id: int = 1,
        cmd_type: int = 2,
        mode: int = 1,
        base_index: int = 0,
        tool_index: int = 0,
        velocity: float = 5.0,
        acceleration: float = 1.0,
        blending: float = 0.1,
        WaitForGripper = False,
        x: float = 0.0,
        y: float = 0.0,
        z: float = 0.0,
        a: float = 0.0,
        b: float = 0.0,
        c: float = 0.0,
    ):
        self.id = cmd_id
        self.type = cmd_type
        self.mode = mode
        self.base_index = base_index
        self.tool_index = tool_index
        self.velocity = velocity
        self.acceleration = acceleration
        self.blending = blending   
        self.WaitForGripper = WaitForGripper     

        self.x = x
        self.y = y
        self.z = z
        self.a = a
        self.b = b
        self.c = c

    def to_xml(self) -> bytes:
        root = ET.Element("RobotCommand")
        root.set("Id", str(self.id))
        root.set("Type", str(self.type))

        move = ET.SubElement(root, "Move")
        move.set("Mode", str(self.mode))
        move.set("BaseIndex", str(self.base_index))
        move.set("ToolIndex", str(self.tool_index))
        move.set("Velocity", str(self.velocity))
        move.set("Acceleration", str(self.acceleration))
        move.set("Blending", str(self.blending))
        move.set("WaitForGripper", str(self.WaitForGripper))

        target = ET.SubElement(move, "Cartesian")
        target.set("X", str(self.x))
        target.set("Y", str(self.y))
        target.set("Z", str(self.z))
        target.set("A", str(self.a))
        target.set("B", str(self.b))
        target.set("C", str(self.c))

        targetAux = ET.SubElement(move, "Cartesian_Aux")
        targetAux.set("X", "0.0")
        targetAux.set("Y", "0.0")
        targetAux.set("Z", "0.0")
        targetAux.set("A", "0.0")
        targetAux.set("B", "0.0")
        targetAux.set("C", "0.0")

        joint = ET.SubElement(move, "Joint")
        joint.set("A1", "0.0")
        joint.set("A2", "0.0")
        joint.set("A3", "0.0")
        joint.set("A4", "0.0")
        joint.set("A5", "0.0")
        joint.set("A6", "0.0")


     


        # WICHTIG → XML in String konvertieren
        xml_body = ET.tostring(root, encoding="utf-8", method="xml").decode("utf-8")

        # WICHTIG → vollständige EthernetKRL Message bauen
        full_message = f'<?xml version="1.0" encoding="UTF-8"?>\n<EthernetKRL>\n{xml_body}\n</EthernetKRL>\n'

        # WICHTIG → mit newline abschließen!
        return full_message.encode("utf-8")
    
    def setTool(self):
        print('tbd')

    def setBase(self):
        print('tbd')

    def PtP(self):
        print('tbd')

    def LIN(self):
        print('tbd')

    def CIRC(self):
        print('tbd')

    def __repr__(self):
        return (
            f"MovePrimitive(Type={self.type}, Mode={self.mode}, "
            f"Base={self.base_index}, Tool={self.tool_index}, "
            f"Vel={self.velocity}, Pose=({self.x}, {self.y}, {self.z}, "
            f"{self.a}, {self.b}, {self.c}))"
        )

### class KukaClient ###

- Manages TCP/IP communication with the KUKA controller (EKI interface)
- Stores and sends MovePrimitive objects

- Provides methods for:

    - connect() – establish connection
    - disconnect() – close connection
    - add_move() – store a motion command
    - send_move() – send one motion command
    - send_all_moves() – send all stored commands

In [3]:
class KukaClient:
    """
    Manages TCP/IP communication with the KUKA controller via EKI.
    Can store and send MovePrimitive objects.
    """

    def __init__(self, ip: str, port: int):
        self.ip = ip
        self.port = port
        self.socket = None
        self.client_socket = None
        self.connected = False
        self.moves: list[MovePrimitive] = []

    # --------------------------
    # Establish connection
    # --------------------------
    def connect(self):
        try:
            # TCP Client Socket
            self.socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            # optional: Timeout
            #self.socket.settimeout(5.0)
            
            # Verbindung zum Roboter aufbauen
            self.socket.connect((self.ip, self.port))
            print(f"Connected to KUKA at {self.ip}:{self.port}")
            self.connected = True
        except Exception as e:
            print(f"Connection failed: {e}")
            self.connected = False

    # --------------------------
    # Close connection
    # --------------------------
    def disconnect(self):
        if self.socket:
            self.socket.close()
            print("Connection closed.")
        self.connected = False

    # --------------------------
    # Add a MovePrimitive to the list
    # --------------------------
    def add_move(self, move: MovePrimitive):
        self.moves.append(move)
        print(f"Move added: {move}")

    # --------------------------
    # Send a single MovePrimitive
    # --------------------------
    def send_move(self, move: MovePrimitive):
        if not self.connected:
            print("Not connected - please call connect() first.")
            return

        xml_data = move.to_xml()
        try:
            # -------------- UDP --------------------------
            # self.socket.sendto(xml_data, (self.ip, self.port))

            # --------------- TCP -------------------------
            self.socket.sendall(xml_data)

            print(f"Move sent:\n{xml_data.decode()}")

            # Optionally receive response from KUKA
            
            self.socket.settimeout(1.0)
            try:
                data = self.socket.recv(4096)
                if data:
                    print(f"Response from KUKA:\n{data.decode()}")
            except socket.timeout:
                print("(No response received - KUKA may be in RECEIVE mode only.)")
            
        except Exception as e:
            print(f"Send error: {e}")

    # --------------------------
    # Send all stored MovePrimitives
    # --------------------------
    def send_all_moves(self):
        if not self.connected:
            print("Not connected - please call connect() first.")
            return

        for move in self.moves:
            self.send_move(move)

In [5]:
kuka = KukaClient(ip="10.181.116.41", port=54602)
kuka.connect()

Connected to KUKA at 10.181.116.41:54602


In [7]:
print(kuka.socket)

<socket.socket fd=65, family=2, type=1, proto=0, laddr=('10.181.116.200', 35874), raddr=('10.181.116.41', 54602)>


In [13]:
kuka.disconnect()

Connection closed.


In [14]:
Move = MovePrimitive(cmd_type=1, mode=2, x=340.0, y=0.0, z=630.0, a=55.0, b=77.0, c=55.0)
kuka.send_move(Move)

Move sent:
<?xml version="1.0" encoding="UTF-8"?>
<EthernetKRL>
<RobotCommand Id="1" Type="1"><Move Mode="2" BaseIndex="0" ToolIndex="0" Velocity="5.0" Acceleration="1.0" Blending="0.1" WaitForGripper="False"><Cartesian X="340.0" Y="0.0" Z="630.0" A="55.0" B="77.0" C="55.0" /><Cartesian_Aux X="0.0" Y="0.0" Z="0.0" A="0.0" B="0.0" C="0.0" /><Joint A1="0.0" A2="0.0" A3="0.0" A4="0.0" A5="0.0" A6="0.0" /></Move></RobotCommand>
</EthernetKRL>

(No response received - KUKA may be in RECEIVE mode only.)


In [8]:
Move1 = MovePrimitive(cmd_type=1, mode=2, x=335.0, y=0.0, z=600.0, a=75.0, b=77.0, c=55.0)
kuka.add_move(Move1)
#Move2 = MovePrimitive(cmd_type=1, mode=2, x=240.0, y=0.0, z=580.0, a=70.0, b=80.0, c=85.0)
#kuka.add_move(Move2)
kuka.send_all_moves()


Move added: MovePrimitive(Type=1, Mode=2, Base=0, Tool=0, Vel=5.0, Pose=(335.0, 0.0, 600.0, 75.0, 77.0, 55.0))
Move sent:
<?xml version="1.0" encoding="UTF-8"?>
<EthernetKRL>
<RobotCommand Id="1" Type="1"><Move Mode="2" BaseIndex="0" ToolIndex="0" Velocity="5.0" Acceleration="1.0" Blending="0.1" WaitForGripper="False"><Cartesian X="300.0" Y="0.0" Z="600.0" A="75.0" B="77.0" C="55.0" /><Cartesian_Aux X="0.0" Y="0.0" Z="0.0" A="0.0" B="0.0" C="0.0" /><Joint A1="0.0" A2="0.0" A3="0.0" A4="0.0" A5="0.0" A6="0.0" /></Move></RobotCommand>
</EthernetKRL>

(No response received - KUKA may be in RECEIVE mode only.)
Move sent:
<?xml version="1.0" encoding="UTF-8"?>
<EthernetKRL>
<RobotCommand Id="1" Type="1"><Move Mode="2" BaseIndex="0" ToolIndex="0" Velocity="5.0" Acceleration="1.0" Blending="0.1" WaitForGripper="False"><Cartesian X="320.0" Y="0.0" Z="600.0" A="75.0" B="77.0" C="55.0" /><Cartesian_Aux X="0.0" Y="0.0" Z="0.0" A="0.0" B="0.0" C="0.0" /><Joint A1="0.0" A2="0.0" A3="0.0" A4="0.0" A